In [ ]:
# Conformal Prediction Example1

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
import openpyxl
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import roc_curve, auc

In [ ]:
# Read input data

In [2]:
data =  pd.read_csv('problem6.csv')
data.head()

,ROA,P/B,Class
0,1.67,1.02,1
1,2.87,0.86,1
2,3.16,0.93,1
3,1.16,1.34,1
4,1.67,0.79,1


In [ ]:
# 驗證是否讀取正確

In [3]:
data.describe()

,ROA,P/B,Class
count,1035.000000,1035.000000,1035.000000
mean,1.706000,2.226464,1.158454
std,2.299725,1.694491,0.365343
min,-7.440000,0.040000,1.000000
25%,0.475000,1.155000,1.000000
50%,1.570000,1.730000,1.000000
75%,2.800000,2.665000,1.000000
max,19.790000,15.200000,2.000000


In [ ]:
# 資料筆數

In [4]:
totalsample=data.shape[0]
print(totalsample)

1035


In [ ]:
# 資料分割

In [5]:
X = data.iloc[:,0:1].values
print(type(X))
Y = data['Class'].values
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.25,random_state = 0)

<class 'numpy.ndarray'>


In [ ]:
# 資料正規化

In [6]:
stdscaler = StandardScaler()
X_train = stdscaler.fit_transform(X_train)
X_test = stdscaler.transform(X_test)